In [2]:
import pandas as pd

In [10]:
blogs = pd.read_csv('blogs.csv')
blogs['num_words'] = blogs['text'].str.split().str.len()
blogs['num_sentences'] = blogs['text'].str.split('.').str.len()

filtered_blogs = blogs[(blogs['num_words'] > 7) & (blogs['tag'] == 'p')]
filtered_blogs.head()

,Unnamed: 0,text,tag,paragraph,article,num_words,num_sentences
1,1,One of the most helpful things in my own recov...,p,NaN,/blog/nocd-support-groups-finding-help-and-hop...,56,5
2,2,There is something powerful about knowing that...,p,NaN,/blog/nocd-support-groups-finding-help-and-hop...,60,4
3,3,Support is a key piece of your recovery journe...,p,NaN,/blog/nocd-support-groups-finding-help-and-hop...,78,4
5,5,Support groups may help you realize that you a...,p,You are not alone,/blog/nocd-support-groups-finding-help-and-hop...,85,5
6,6,This is one of the main reasons I continue to ...,p,You are not alone,/blog/nocd-support-groups-finding-help-and-hop...,77,4


In [13]:
def get_text(df):
    for index, row in df.iterrows():
        yield row['text']

In [15]:
passages = get_text(filtered_blogs)

for i, passage in enumerate(passages):
    print(passage + '\n')
    if i == 2: break

One of the most helpful things in my own recovery journey has been hearing about other people’s experiences with OCD. This might be because we relish in stories of triumph and are drawn toward people with shared experiences. I think this is true for many things in life. Personally, I learn and grow from these stories.

There is something powerful about knowing that someone else has walked the same path as you and that they have not only survived it, but possibly even thrived. I love hearing about the determination and grit of others who have faced similar obstacles. These stories of hope often inspired me to keep going, even when I didn’t think I could. 

Support is a key piece of your recovery journey. That’s why at NOCD, we provide a safe space for people in the OCD community and their families to share their experiences in our virtual support groups, which are available to members doing exposure and response prevention (ERP) therapy with NOCD Therapy. There are over 25 different gro

In [17]:
!pip install transformers[torch]

     |████████████████████████████████| 4.9 MB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 67.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 65.2 MB/s eta 0:00:01
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)
     |████████████████████████████████| 772 kB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 776.3 MB 227 bytes/s a 0:00:01     |██████████▋                     | 256.7 MB 54.6 MB/s eta 0:00:10     |██████████▉                     | 262.9 MB 54.6 MB/s eta 0:00:10     |█████████████                   | 317.5 MB 37.3 MB/s eta 0:00:13


In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'doc2query/msmarco-t5-base-v1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

ModuleNotFoundError: No module named 'torch'

In [20]:
import torch
print(torch.__version__)

1.12.1+cu102
